In [4]:
# Config Variables

# Define UCI Datasets to pull
# https://archive.ics.uci.edu/ml/datasets.php
uci_datasets = [165,186,294]

# Use all data (vs just training data) for OLS to determine co-efficient sign
ols_use_all_data = True

# Training set sizes to use (only training set sizes < 60% of total dataset size will be used)
#training_set_sizes = [10, 40, 160, 640, 2560, 10240, 40960]
training_set_sizes = [10, 40, 160, 610, 2560, 10240, 40960]
chain_lengths = [20000, 20000, 100000, 100000, 100000, 100000, 100000]

# Number of etas (prior standard deviation) to test between 1e-04 and 1e05 (recommend 20)
# Limits at 0 and infinity are calculated separately
# Note: In practice, sigma^2 (likelihood variance) is set to 1/eta^2 and
#       lambda (penalty) is set to 1/eta^4 
num_etas = 20

# Number of folds for cross-validation (set to 300 in 2018 paper, recommend testing with 20)
cv_folds = 20

# Overwrite model if it's already saved to file
overwrite_model = False

In [5]:
# Load necessary libraries and helper functions

# import half_ridge functions from helpers/half_ridge.py
from helpers.half_ridge import *

# import testing functions from helpers/testing_functions.py
from helpers.testing_functions import *

# import dataset class
from helpers.model import Model

# import os and pickle
import os
import pickle
 
# Pandas and numpy for data manipulation
import pandas as pd
import numpy as np
np.random.seed(42)

import time

In [6]:
# Load the ICU dataset, normalize the data, and split into training, validation, and test sets

# Loop over each dataset, loaded from file and formatted
for dataset in uci_datasets:
    model = Model()
    model.import_from_file('uci_ml_datasets/ucirepo_' + str(dataset) + '.csv')
    model.format_data()
    training_set_values = model.set_up_training_values(training_set_sizes, .6)

    # Check if a file exists for the dataset in the model_results folder
    if os.path.isfile(f"model_results/ucirepo_{dataset}.csv") and not overwrite_model:
        print(f"Model already exists for dataset {dataset}. Skipping...")
        continue

    # Print the training set values for this dataset
    print(f"Training Set Values: {training_set_values}")
    
    # Initialize the comparison results
    binary_comparisons = {}

    # Determine the signs of the coefficients using OLS (all data)
    if ols_use_all_data:
        ols_coefficients = model.get_ols_coefficients(True)
        weight_signs = np.sign(list(ols_coefficients.values()))

    # loop over the training set values
    for training_set_index, training_set_value in enumerate(training_set_values):

        # Generate cross-validation indices
        cv = model.cv_indexing(cv_folds, training_set_value)

        # Initialize the comparison results for this training set value
        binary_comparisons[training_set_value] = {}

        # Vary eta values from 10^-4 to 10^5, including 0 and inf, and loop over them
        etas = np.logspace(-4, 5, num=num_etas)
        etas = np.insert(etas, 0, 0)
        etas = np.append(etas, np.inf)
        for eta in etas:

            # Initialize the comparison results for this eta
            binary_comparisons[training_set_value][eta] = []

            # Crossvalidate over the folds for a specific eta
            for i in range(cv_folds):

                # Print info about this iteration
                print(f"Training Set Value: {training_set_value}, Eta: {eta}, Fold: {i} of {range(cv_folds)}")

                # Get training and test indices for the i-th fold
                train_indices = np.where(cv[i] == 1)[0]
                test_indices = np.where(cv[i] == 0)[0]
                
                # Split the data
                X_train, X_test = model.X.iloc[train_indices], model.X.iloc[test_indices]
                y_train, y_test = model.y.iloc[train_indices], model.y.iloc[test_indices]

                # Get OLS coefficients for the training data
                ols_coefficients = model.get_ols_coefficients(False, X_train, y_train)

                # Determine the signs of the coefficients using OLS on only the training data
                if not ols_use_all_data:
                    weight_signs = np.sign(list(ols_coefficients.values()))

                # Run the half ridge regression with mcmc
                #half_ridge_trace = half_ridge(X_train, y_train, model.ols_coefficients, prior_eta=eta)
                #var_dict = {}
                #for variable in half_ridge_trace.posterior:
                #    var_dict[variable] = half_ridge_trace.posterior[variable].values.flatten()
                # Results into a dataframe
                #var_weights = pd.DataFrame(var_dict)
                # Means for all the weights
                #var_means_array = var_weights.mean(axis=0)
                # Convert the dataframe var_means_array to a dictionary with the column names as keys
                #half_ridge_coefficients = {col: var_means_array[col] for col in var_weights.columns}
                
                # Run the half ridge regression with rejection sampling
                start_time = time.time()
                half_ridge_coefficients = half_ridge_rejection_sampling(weight_signs, ols_coefficients, X_train, y_train, eta, chain_lengths[training_set_index])
                end_time = time.time()
                duration = end_time - start_time
                print(f"      Half ridge regression rejection sampling took {duration} seconds")

                # Calculate the binary comparisons for the models
                sorted_half_ridge_coefficients = {k: half_ridge_coefficients[k] for k in sorted(half_ridge_coefficients)}
                sorted_half_ridge_coefficients['intercept'] = 0
                print(f"      half-ridge coefficients: {sorted_half_ridge_coefficients}")
                binary_comparison = calculate_binary_comparison(sorted_half_ridge_coefficients, X_test, y_test)
                binary_comparisons[training_set_value][eta].append(binary_comparison)
                print(f"      binary comparison: {binary_comparison}")

                # Calculate SSE for the half ridge coefficients
                #half_ridge_sse = calculate_sse(sorted_half_ridge_coefficients, X_test, y_test)
                #binary_comparisons[training_set_value][eta].append(half_ridge_sse)
                #print(f"half ridge sse: {half_ridge_sse}")

    
    with open('model_results/ucirepo_' + str(dataset) + '.csv', 'wb') as file:
        pickle.dump(binary_comparisons, file)


Model already exists for dataset 165. Skipping...
Model already exists for dataset 186. Skipping...
Training Set Values: [10, 40, 160, 610, 2560]
Training Set Value: 10, Eta: 0.0, Fold: 0 of range(0, 20)
Running half-ridge regression...
      Half ridge regression rejection sampling took 3.1948089599609375e-05 seconds
      half-ridge coefficients: {'AP': 0.7978845608028654, 'AT': -0.7978845608028654, 'RH': -0.7978845608028654, 'V': -0.7978845608028654, 'intercept': 0}
      binary comparison: 0.722
Training Set Value: 10, Eta: 0.0, Fold: 1 of range(0, 20)
Running half-ridge regression...
      Half ridge regression rejection sampling took 1.2159347534179688e-05 seconds
      half-ridge coefficients: {'AP': 0.7978845608028654, 'AT': -0.7978845608028654, 'RH': -0.7978845608028654, 'V': -0.7978845608028654, 'intercept': 0}
      binary comparison: 0.718
Training Set Value: 10, Eta: 0.0, Fold: 2 of range(0, 20)
Running half-ridge regression...
      Half ridge regression rejection samplin

/Library/WebServer/Sites/SURF/bayesian_modeling/helpers/half_ridge.py:113: RuntimeWarning: Mean of empty slice
  post_weights = np.nanmean(samples1, axis=0)


      binary comparison: 0.496
Training Set Value: 10, Eta: 16.23776739188721, Fold: 6 of range(0, 20)
Running half-ridge regression...
      Half ridge regression rejection sampling took 0.015921831130981445 seconds
      half-ridge coefficients: {'AP': -0.15504361981469506, 'AT': -0.2569452533650496, 'RH': -0.03981640429460689, 'V': -0.41651831822483093, 'intercept': 0}


/Library/WebServer/Sites/SURF/bayesian_modeling/helpers/half_ridge.py:113: RuntimeWarning: Mean of empty slice
  post_weights = np.nanmean(samples1, axis=0)


      binary comparison: 0.744
Training Set Value: 10, Eta: 16.23776739188721, Fold: 7 of range(0, 20)
Running half-ridge regression...
      Half ridge regression rejection sampling took 0.025487899780273438 seconds
      half-ridge coefficients: {'AP': 0.0073785571278010605, 'AT': -0.9890173044049643, 'RH': -0.08655452914809984, 'V': -0.05572203827679158, 'intercept': 0}
      binary comparison: 0.587
Training Set Value: 10, Eta: 16.23776739188721, Fold: 8 of range(0, 20)
Running half-ridge regression...
      Half ridge regression rejection sampling took 0.012140989303588867 seconds
      half-ridge coefficients: {'AP': -0.08627937935129688, 'AT': -0.9083084521394441, 'RH': -0.11050987121758056, 'V': -0.0626376008866306, 'intercept': 0}


...

      binary comparison: 0.615
Training Set Value: 610, Eta: 33598.18286283774, Fold: 0 of range(0, 20)
Running half-ridge regression...
      Half ridge regression rejection sampling took 0.01824498176574707 seconds
      half-ridge coefficients: {'AP': 0.030116738106086487, 'AT': -0.8448185006339267, 'RH': -0.1283822769312036, 'V': -0.18295125446118543, 'intercept': 0}
      binary comparison: 0.714
Training Set Value: 610, Eta: 33598.18286283774, Fold: 1 of range(0, 20)
Running half-ridge regression...
      Half ridge regression rejection sampling took 0.009577035903930664 seconds
      half-ridge coefficients: {'AP': 0.013099701199262188, 'AT': -0.8848954993067001, 'RH': -0.12576848871686783, 'V': -0.17319312697358008, 'intercept': 0}
      binary comparison: 0.715
Training Set Value: 610, Eta: 33598.18286283774, Fold: 2 of range(0, 20)
Running half-ridge regression...
      Half ridge regression rejection sampling took 0.0096588134765625 seconds
      half-ridge coefficients: {'A

/Library/WebServer/Sites/SURF/bayesian_modeling/helpers/half_ridge.py:113: RuntimeWarning: Mean of empty slice
  post_weights = np.nanmean(samples1, axis=0)


      binary comparison: 0.604
Training Set Value: 610, Eta: 100000.0, Fold: 0 of range(0, 20)
Running half-ridge regression...
      Half ridge regression rejection sampling took 0.009668827056884766 seconds
      half-ridge coefficients: {'AP': 0.03011674351613852, 'AT': -0.8448184899945029, 'RH': -0.1283822675076998, 'V': -0.182951262974185, 'intercept': 0}
      binary comparison: 0.72
Training Set Value: 610, Eta: 100000.0, Fold: 1 of range(0, 20)
Running half-ridge regression...
      Half ridge regression rejection sampling took 0.012100934982299805 seconds
      half-ridge coefficients: {'AP': 0.013099700107457143, 'AT': -0.8848954933669814, 'RH': -0.12576848074939678, 'V': -0.17319311836403184, 'intercept': 0}
      binary comparison: 0.698
Training Set Value: 610, Eta: 100000.0, Fold: 2 of range(0, 20)
Running half-ridge regression...
      Half ridge regression rejection sampling took 0.009589910507202148 seconds
      half-ridge coefficients: {'AP': 0.04289919091254487, 'AT

/Library/WebServer/Sites/SURF/bayesian_modeling/helpers/half_ridge.py:113: RuntimeWarning: Mean of empty slice
  post_weights = np.nanmean(samples1, axis=0)


      binary comparison: 0.589
Training Set Value: 610, Eta: inf, Fold: 0 of range(0, 20)
Running half-ridge regression...
      Half ridge regression rejection sampling took 7.152557373046875e-06 seconds
      half-ridge coefficients: {'AP': 0.030205514987042294, 'AT': -0.8447530061852376, 'RH': -0.12838053142657818, 'V': -0.18298925594521975, 'intercept': 0}
      binary comparison: 0.726
Training Set Value: 610, Eta: inf, Fold: 1 of range(0, 20)
Running half-ridge regression...
      Half ridge regression rejection sampling took 5.9604644775390625e-06 seconds
      half-ridge coefficients: {'AP': 0.013109122176267297, 'AT': -0.8843839222860188, 'RH': -0.12539733658816507, 'V': -0.17344841507658043, 'intercept': 0}
      binary comparison: 0.697
Training Set Value: 610, Eta: inf, Fold: 2 of range(0, 20)
Running half-ridge regression...
      Half ridge regression rejection sampling took 4.76837158203125e-06 seconds
      half-ridge coefficients: {'AP': 0.04274337437000499, 'AT': -0.8